# Obtendo e manipulando dados

### Básico sobre pastas e arquivos

Este módulo fornece uma maneira para interagir com o sistema operacional. 

- Se você quer apenas ler ou escrever um arquivo veja **open()**, 
- se você quer manipular caminhos, veja o **os.path** módulo, 
- e se você quer ler todas as linhas em todos os arquivos na linha de comando veja o **fileinput** módulo. 
- Para criar arquivos e diretórios temporários veja o **tempfile** módulo, e para manipulação de arquivos e diretórios de alto nível veja o **shutil** módulo.

Vamos ver algumas operações simples.

#### os.environ 
Obter diversas informações do seu sistema e todas elas estão no formato de dicionário.

In [ ]:
import os


os.environ

#### os.getcwd()
Obter o diretorio de trabalho atual

#### os.chdir()
Mudar para um diretorio expecífico

#### os.mkdir()
Cria um diretorio

#### os.listdir()
Listar o conteúdo de um diretorio

#### os.rename()
Renomer diretorio ou arquivo

#### os.rmdir()
Remover o diretorio

#### Trabalhando com paths

### Trabalhando com arquivos JSON (retorno de APIs)

O formato JSON é utilizado para estruturar dados em formato de texto e permitir a troca de dados entre aplicações de forma simples, leve e rápida. Por isso é tão importante saber como é estruturado e as principais diferenças com o modelo XML.

JSON, que significa JavaScript Object Notation, é uma formatação utilizada para estruturar dados em formato de texto e transmiti-los de um sistema para outro, como em aplicações cliente-servidor ou em aplicativos móveis.

A especificação JSON surgiu por volta do ano 2000. Atualmente, esse formato é suportado por diversos tipos de linguagem de programação, além de ser uma alternativa mais leve que o modelo XML.

Os dados contidos em um arquivo no formato JSON devem ser estruturados por meio de uma coleção de pares com nome e valor ou ser uma lista ordenada de valores. Seus elementos devem conter:

* chave: corresponde ao identificador do conteúdo. Por isso, deve ser uma string delimitada por aspas;
* valor: representa o conteúdo correspondente e pode conter os seguintes tipos de dados: string, array, object, number, boolean ou null.

Além da terminação .json em todos os arquivos que utilizam esse formato, os dados armazenados devem seguir uma notação específica, ou seja, precisam ser organizados com os seguintes elementos básicos:

* chaves { } para delimitar os objetos e obrigatórias para iniciar e encerrar o conteúdo;
* colchetes [ ] para indicar um array;
* dois pontos : para separar a chave e seu valor correspondente;
* vírgula , para indicar a separação entre os elementos

Fonte: https://rockcontent.com/br/blog/json/

Quase sempre, ao realizarmos acessos na Internet, acabamos por usar também uma outra biblioteca: a requests.
Como diz o site oficial: Requests é uma biblioteca HTTP para Python simples e elegante, feita para seres humanos.

Fonte: https://docs.python-requests.org/pt_BR/latest/user/quickstart.html

Para podermos entender um pouco de como acessar dados desses tipos, vamos utilizar um site que disponibiliza uma API para testes. Esse site é o http://makeup-api.herokuapp.com/

In [ ]:
import json
import requests

In [ ]:
brand = 'maybelline'
url ='http://makeup-api.herokuapp.com/api/v1/products.json?brand=%s' % (brand)

requisicao = requests.get(url)


In [ ]:
requisicao.status_code

In [ ]:
requisicao.text

In [ ]:
dados = json.loads(requisicao.text)
dados

In [ ]:
import pandas as pd

dados = pd.json_normalize(dados)

In [ ]:
dados.info()

In [ ]:
dados.head()

### Coletando dados na web - Beautiful Soup

Beautiful Soup é uma biblioteca Python de extração de dados de arquivos HTML e XML. Ela funciona com o seu interpretador (parser) favorito a fim de prover maneiras mais intuitivas de navegar, buscar e modificar uma árvore de análise (parse tree). Ela geralmente economiza horas ou dias de trabalho de programadores ao redor do mundo.


https://www.crummy.com/software/BeautifulSoup/bs4/doc.ptbr/

Para poder interagir com essa biblioteca e ter proveito, um básico de html ajuda muito.

#### Primeiro exemplo: Buscando a lista de capitais do Brasil por Área 

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
url = 'https://pt.wikipedia.org/wiki/Lista_de_capitais_do_Brasil_por_%C3%A1rea'
wiki_page = requests.get(url)

In [ ]:
soup = BeautifulSoup(wiki_page.text, 'html.parser')

In [ ]:
tabelas_exitentes = soup.find_all('table', class_='wikitable sortable')
tabelas_exitentes

In [ ]:
tabela_capitais = soup.find('table', class_='wikitable sortable')
tabela_capitais

In [ ]:
#gerando a lista em colunas
lista_posicao = []
lista_estado = []
lista_cod = []
lista_capital = []
lista_municipio = []

for linha in tabela_capitais.findAll("tr"): #para tudo que estiver em <tr>
    celula = linha.findAll('td') #variável para encontrar <td>
    if len(celula)==5: #número de colunas
        lista_posicao.append(celula[0].find(text=True)) #iterando sobre cada linha
        lista_estado.append(celula[1].find(text=True))
        lista_cod.append(celula[2].find(text=True))
        lista_capital.append(celula[3].find('a').text)
        lista_municipio.append(celula[4].find(text=True))

In [ ]:
df_lstMunic = pd.DataFrame(columns=['Posição'])

df_lstMunic['Posição'] = lista_posicao
df_lstMunic['Estado'] = lista_estado
df_lstMunic['Código/IBGE'] = lista_cod
df_lstMunic['Capital'] = lista_capital
df_lstMunic['Área'] = lista_municipio

df_lstMunic

### Baixando arquivos

In [ ]:
url = 'http://xkcd.com' # url inicial
os.chdir(r'/home/valli/repos/ensino/turma-ead-20220301')
os.makedirs('xkcd', exist_ok=True) # armazena as tirinhas em ./xkcd 

while not url.endswith('#'):
    # Faz download da página.
    print(f'Downloading page {url}...')

    res = requests.get(url)

    soup = BeautifulSoup(res.text)

    # Encontra o URL da imagem da tirinha.
    comicElem = soup.select('#comic img')
    if comicElem == []:
        print('Could not find comic image.')
    else:
        comicUrl = 'http:' + comicElem[0].get('src')
        # Faz o download da imagem.
        print(f'Downloading image {comicUrl}...')
        res = requests.get(comicUrl)
        res.raise_for_status()


    with open(os.path.join('xkcd', os.path.basename(comicUrl)), 'wb') as imageFile:
        for chunk in res.iter_content(100000):
            imageFile.write(chunk)
            
    # Obtém o url do botão Prev. print('Done.')
    prevLink = soup.select('a[rel="prev"]')[0]
    url = 'http://xkcd.com' + prevLink.get('href')
print('Done')

### Buscando salários do Glassdoor - Exemplo Leon Sólon

In [ ]:
headers = { 'user-agent' : 'Mozilla/5.0' }
url_site = 'https://www.glassdoor.com.br/Sal%C3%A1rios/cientista-de-dados-sal%C3%A1rio-SRCH_KO0,18.htm'

resposta = requests.get(url_site, headers= headers)

resposta.text

In [ ]:
soup = BeautifulSoup(resposta.text, 'html.parser')
soup

In [ ]:
empresas = soup.find_all('h3', {'data-test' : 'salaries-list-item-0-employer-name'})
empresas[0]

In [ ]:
empresas[0].contents

In [ ]:
empresas[0].contents[1].text

In [ ]:
import re

In [ ]:
empresas = soup.find_all('h3', {'data-test' : re.compile('salaries-list-item-.*-employer-name')})

In [ ]:
for empresa in empresas:
    print(empresa.find('a').text)

In [ ]:
salarios = soup.find_all('div', {'data-test' : re.compile('salaries-list-item-.*-salary-info')})

In [ ]:
salarios

In [ ]:
dados_salariais = []
for empresa, salario in zip(empresas,salarios):
    nome_empresa = (empresa.find('a').text)
    salario_empresa = (salario.find('h3').text.replace('.','').replace('R$\xa0', ''))

    dados_salariais.append((nome_empresa, salario_empresa))

In [ ]:
dados_salariais
df = pd.DataFrame(dados_salariais, columns=['Empresa','Salario'])
df